# Agent Benchmarking: Search + Calculator

Here we go over how to benchmark performance of an agent on tasks where it has access to a calculator and a search tool.

It is highly reccomended that you do any evaluation/benchmarking with tracing enabled. See [here](https://langchain.readthedocs.io/en/latest/tracing.html) for an explanation of what tracing is and how to set it up.

In [1]:
# Comment this out if you are NOT using tracing
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

## Loading the data
First, let's load the data.

In [2]:
from langchain.evaluation.loading import load_dataset
dataset = load_dataset("agent-search-calculator")

Found cached dataset json (/Users/wfh/.cache/huggingface/datasets/LangChainDatasets___json/LangChainDatasets--agent-search-calculator-8a025c0ce5fb99d2/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

## Setting up a chain
Now we need to load an agent capable of answering these questions.

In [3]:
from langchain.llms import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.agents import AgentType

tools = load_tools(['serpapi', 'llm-math'], llm=OpenAI(temperature=0))
agent = initialize_agent(tools, OpenAI(temperature=0), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)


## Make a prediction

First, we can make predictions one datapoint at a time. Doing it at this level of granularity allows use to explore the outputs in detail, and also is a lot cheaper than running over multiple datapoints

In [4]:
print(dataset[0]['question'])
agent.run(dataset[0]['question'])

How many people live in canada as of 2023?


'38,660,121'

## Make many predictions
Now we can make predictions

In [5]:
agent.run(dataset[4]['question'])

For input: 38^.23
For input: 38^.23
Attempting to evaluate [38**.23] or ```text
38**.23
```
Got an output [2.3086081644669734]


'The total number of points scored in the 2023 Super Bowl raised to the .23 power is 2.3086081644669734.'

In [28]:
predictions = []
predicted_dataset = []
error_dataset = []
for data in dataset:
    new_data = {"input": data["question"], "answer": data["answer"]}
    try:
        predictions.append(agent(new_data))
        predicted_dataset.append(new_data)
    except Exception as e:
        predictions.append({"output": str(e), **new_data})
        error_dataset.append(new_data)

For input: 41^.43
For input: 41^.43
Attempting to evaluate [41**.43] or ```text
41**.43
```
Got an output [4.9373857399466665]
For input: dua lipa's boyfriend age raised to the .43 power
For input: dua lipa's boyfriend age raised to the .43 power
Attempting to evaluate [2.43 ** 2.43] or ```text
2.43 ** 2.43
```
Got an output [8.650142158924355]
For input: 38^.23
For input: 38^.23
Attempting to evaluate [38**.23] or ```text
38**.23
```
Got an output [2.3086081644669734]
For input: Total number of points scored in the 2023 Super Bowl
For input: Total number of points scored in the 2023 Super Bowl
For input: 38 - 20
For input: 38 - 20
Attempting to evaluate [38 - 20] or ```text
38 - 20
```
Got an output [18]
For input: 153 raised to .1312
For input: 153 raised to .1312
Attempting to evaluate [153 ** .1312] or ```text
153 ** .1312
```
Got an output [1.9347796717823205]
For input: A$AP Rocky's height raised to the .13 power
For input: A$AP Rocky's height raised to the .13 power
Attempting t

## Evaluate performance
Now we can evaluate the predictions. The first thing we can do is look at them by eye.

In [21]:
predictions[0]

{'input': 'how far is it from paris to boston in miles',
 'answer': 'approximately 3,435 mi',
 'output': 'The shortest distance (air line) between Boston and Paris is 3,437.00 mi (5,531.32 km).'}

Next, we can use a language model to score them programatically

In [22]:
from langchain.evaluation.qa import QAEvalChain

In [29]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(dataset, predictions, question_key="question", prediction_key="output")

We can add in the graded output to the `predictions` dict and then get a count of the grades.

In [30]:
for i, prediction in enumerate(predictions):
    prediction['grade'] = graded_outputs[i]['text']

In [31]:
from collections import Counter
Counter([pred['grade'] for pred in predictions])

Counter({' CORRECT': 6, ' INCORRECT': 4})

We can also filter the datapoints to the incorrect examples and look at them.

In [ ]:
incorrect = [pred for pred in predictions if pred['grade'] == " INCORRECT"]

In [ ]:
incorrect[0]